In [8]:
#Básicos para manipulacion de datos 
import pandas as pd
import numpy as np
#Graficas 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set_theme()
#Optimización multiobjetivo 
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.optimize import minimize
from pymoo.core.population import Population
from pymoo.core.mutation import Mutation

from pymoo.core.population import Population
#Finanzas 
import yfinance as yf
import yesg

#Plugins
from tqdm import tqdm
from itertools import compress
from Plugins import pre_processing
from Plugins import ArchievingStrategies
from Plugins import my_plotting
from Plugins import pymoo_extras
from Plugins import DS

In [9]:
from pymoo.core.sampling import Sampling
class MySampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        #100 individuos (a lo más 2000)
        #problem.n_var la cantidad de activos. 
        X = get_reference_directions("energy", problem.n_var, n_samples, seed=1)
        return X 

In [10]:
from pymoo.core.crossover import Crossover
from pymoo.core.variable import Real, get
# ---------------------------------------------------------------------------------------------------------
# Class
# ---------------------------------------------------------------------------------------------------------
def point_on_triangle(x0, x1, x2):
    s, t = sorted([np.random.random(), np.random.random()])
    return s*x0 + (t-s)*x1 + (1-t)*x2
    
def SPX(x0,x1, x2, epsilon,n_offspring): 
    CoM = 1/3*(x0+x1+x2)
    V1=CoM+(x0-CoM)*(1+epsilon)
    V2=CoM+(x1-CoM)*(1+epsilon)
    V3=CoM+(x2-CoM)*(1+epsilon)
    return [point_on_triangle(x0,x1, x2) for _ in range(n_offspring)]

class SimplexCrossover(Crossover):
    def __init__(self,
                 n_offsprings=3, 
                 epsilon = 0.05,
                 **kwargs):
        super().__init__(3, n_offsprings, **kwargs)
        self.epsilon =  epsilon
        
    def _do(self, problem, X, **kwargs):
        _, n_matings, _ = X.shape 
        Y = np.full_like(X, None)
        for k in range(n_matings):
             # get the first and the second parent
            x0, x1, x2 = X[0, k, :], X[1, k, :], X[2, k, :]
            off_ = SPX(x0, x1, x2, self.epsilon, self.n_offsprings)
            Y[0, k, :] = off_[0]
            Y[1, k, :] = off_[1]
            Y[2, k, :] = off_[2]

        return Y


In [11]:
import numpy as np

from pymoo.core.mutation import Mutation
from pymoo.core.variable import Real, get
from pymoo.operators.repair.bounds_repair import repair_random_init


# ---------------------------------------------------------------------------------------------------------
# Function
# ---------------------------------------------------------------------------------------------------------


def mut_gauss(X, xl, xu, sigma, prob):
    n, n_var = X.shape
    assert len(sigma) == n
    assert len(prob) == n

    Xp = np.full(X.shape, np.inf)

    mut = np.random.random(X.shape) < prob[:, None]

    Xp[:, :] = X

    _xl = np.repeat(xl[None, :], X.shape[0], axis=0)[mut]
    _xu = np.repeat(xu[None, :], X.shape[0], axis=0)[mut]
    sigma = sigma[:, None].repeat(n_var, axis=1)[mut]
    Xp[mut] = np.random.normal(X[mut], sigma * (_xu * _xl))

    Xp = repair_random_init(Xp, X, xl, xu)

    return Xp


# ---------------------------------------------------------------------------------------------------------
# Class
# ---------------------------------------------------------------------------------------------------------


class GaussianMutation(Mutation):

    def __init__(self, sigma=0.1, **kwargs):
        super().__init__(**kwargs)
        self.sigma = Real(sigma, bounds=(0.01, 0.25), strict=(0.0, 1.0))

    def _do(self, problem, X, **kwargs):
        X = X.astype(float)

        sigma = get(self.sigma, size=len(X))
        prob_var = self.get_prob_var(problem, size=len(X))
        print(X)
        Xp = mut_gauss(X, problem.xl, problem.xu, sigma, prob_var)

        return Xp

## Experimento

In [12]:
assets = pd.read_csv( 'Indices/tickers_dowjones.csv', index_col=0)['0'].tolist()
returns= pd.read_csv( 'data/dowjones_returns.csv', index_col=0)
assets_info=pd.read_csv( 'data/dowjones_assets_info.csv', index_col=0)
best_assets=pd.read_csv( 'data/dowjones_best_assets.csv', index_col=0)

In [13]:
PROFITS, RISK, ESG_SCORES = pre_processing.get_final_assets(returns[best_assets.index[:2]], assets_info.loc[best_assets.index[:2]])
portfolio_problem = pymoo_extras.Portfolio_Problem(len(PROFITS), PROFITS, RISK, ESG_SCORES)
from finquant import efficient_frontier
ef = efficient_frontier.EfficientFrontier(pd.Series(PROFITS), pd.DataFrame(RISK), freq=252)
ef_R = ef.efficient_frontier()

In [14]:
from pymoo.termination import get_termination
from pymoo.algorithms.moo.nsga2 import NSGA2
termination = get_termination("n_gen", 2)
nsgaii = NSGA2(pop_size=20,
               sampling=MySampling(), 
               crossover=SimplexCrossover(),
               mutation = GaussianMutation(),
               repair=pymoo_extras.Portfolio_Repair()
               )

In [15]:
eps = np.array([0.01,0.01])
colors = ['rgb(27,158, 119)']

In [16]:
X_nsgaii, F_nsgaii, ESG_nsgaii =pymoo_extras.get_weights_with_pymoo(portfolio_problem, nsgaii, termination)
FA_nsgaii =  pymoo_extras.annualised_portfolio_quantities(F_nsgaii)
frames = [FA_nsgaii]
labels = ['NSGA-II']
fig=my_plotting.plotting_samples_plotly(ef_R, frames, labels, colors)
fig.show()

[[0.35519792 0.64480208]
 [0.28965663 0.71034337]
 [0.48264931 0.51735069]
 [0.74985004 0.25014996]
 [0.5824122  0.4175878 ]
 [0.72749898 0.27250102]
 [0.65940464 0.34059536]
 [0.32944041 0.67055959]
 [0.38672129 0.61327871]
 [0.35950949 0.64049051]
 [0.66735164 0.33264836]
 [0.72652732 0.27347268]
 [0.59874938 0.40125062]
 [0.55419389 0.44580611]
 [0.34788431 0.65211569]
 [0.57412027 0.42587973]
 [0.52413201 0.47586799]
 [0.69405419 0.30594581]
 [0.53455326 0.46544674]
 [0.71536868 0.28463132]
 [0.27598417 0.72401583]]
